In [211]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import string
import nltk
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

from subprocess import check_output

In [212]:
# # I use default email library just for simplicity. 
# # We extract email artificials and content from raw text

# def email_from_string(raw_email):
#     msg = email.message_from_string(raw_email)
    
#     content = []
#     for part in msg.walk():
#         if part.get_content_type() == 'text/plain':
#             content.append(part.get_payload().lower())
            
#     result = {}
#     for key in msg.keys(): 
#         result[key] = msg[key]
#     result["content"] = ''.join(content)
    
#     return result

In [221]:
# Import the email modules we'll need
import glob
import email
import mailparser
from email import policy
from email.parser import BytesParser

path = '../datawe/raw/Email_Classification/*'
email_types = glob.glob(path)
appendFilesData = []
file_raw_data = [] 
for folder in email_types:
    files = glob.glob(folder+"/*.txt")
    email_type = folder.split('\\')[1]
    for name in files:
        try:
            with open(name) as fp:
                raw_data = fp.read()
                file_raw_data.append(raw_data)
                msg = mailparser.parse_from_string(raw_data)
#                 msg = email_from_string(raw_data)
                #appendFilesData.append(fp.read())
                #msg = email.message_from_string(fp)
                #msg = BytesParser(policy=policy.default).parse(fp)
                #richest = ''.join(msg.get_body(preferencelist=('plain', 'html')).get_content().splitlines(keepends=True)[:])
                appendFilesData.append({
                    "to":msg.to,
                    "from":msg.from_,
                    "subject":msg.subject,
                    "date":msg.date,
#                     "sent":msg["Sent"],
#                     "importance":msg["Importance"],
                    "content":raw_data,
                    "class_to_exec":email_type,
                })
         
        except IOError as exc:
            print('Exception')

In [222]:
data = pd.DataFrame(appendFilesData)
#data['length'] = data['content'].apply(len)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 6 columns):
class_to_exec    30 non-null object
content          30 non-null object
date             0 non-null object
from             30 non-null object
subject          30 non-null object
to               30 non-null object
dtypes: object(6)
memory usage: 1.5+ KB


In [223]:
data["class_to_exec"].value_counts()

Enquiry         11
Booking          9
Payment          5
Cancellation     5
Name: class_to_exec, dtype: int64

### removing punctuatins and numbers

In [216]:
# def pre_process(text):
#     text = text.translate(str.maketrans('', '', string.punctuation))
#     text = [word for word in text.split() if word.lower() not in stopwords.words('english')]
#     text = [word for word in text if word.isalnum()]
#     words = ""
#     for i in text:
#             stemmer = SnowballStemmer("english")
#             words += (stemmer.stem(i))+" "
#     return words

### Cleaning(punctuations, alpha charachters) and lemantizing 

In [217]:
def clean(doc):
    words_to_exclude = set(stopwords.words('english'))
    exclude = set(string.punctuation)
    
    tagged_sentence = nltk.tag.pos_tag(doc.split())
    edited_sentence = ' '.join([word for word,tag in tagged_sentence if tag == 'NNP' or tag == 'NNPS']) # or tag == 'NNPS'

    word_free = " ".join([i for i in doc.lower().split() if i not in edited_sentence])
    punc_free = ''.join(ch for ch in word_free if ch not in exclude)
    alpha_free = " ".join(word for word in punc_free.split() if word.isalpha())
    stemmer = SnowballStemmer("english").stem
    stem_free = " ".join(stemmer(stem) for stem in alpha_free.split())
    lemma = WordNetLemmatizer()
    normalized = " ".join(lemma.lemmatize(word) for word in alpha_free.split())
    print(edited_sentence)

    return normalized

# def clean_data(data):
#     return [clean(doc).split(' ') for doc in data]

In [218]:
# textFeatures[19]

In [210]:
textFeatures = data['content'].copy()
textFeatures = textFeatures.apply(clean)
print(len(textFeatures))
vectorizer = TfidfVectorizer("english", smooth_idf=False, sublinear_tf=False, analyzer='word')
features = vectorizer.fit_transform(textFeatures)

features_train, features_test, labels_train, labels_test = train_test_split(features, data['class_to_exec'], test_size=0.2, random_state=111)


Rohit, Taj Gateway Ganges December
Dear Sir/Madam Re: Taj Taj Palace Hotel Taj Swarna Amritsar Taj Fort Aguada Goa. My Husband (Amrit Singh) Taj General Please
Hotel/Resort Arun Gadamshetty P Department Manufacturing Department Contact No Taj Madikeri Coorg, Coorg Room Type Charm Check Date-Time Dec PM Check Date-Time Dec PM No Honeymoon Traveller Details: Arun Gadamshetty Relation Age SHWETA GADAMSHETTY Relation Spouse Age No Please
Dear Gampa Greetings Lerner Group Please CPAI Taj Mahal, New Delhi Dec Kindly
Reservation Dept, Greetings Kuala Lumpur! DELUXE DELIGHT ROOM QUEEN BED WITH BREAKFAST Check \u2013 DECEMBER Check DECEMBER Hope



Hi Please
Dear Kindly Cancel
Mr. Athalye, Mr. Uday Athalye Taj Chandigarh. Cancellation Number: Taj Hotels, Resorts Palaces. Please
Dear Team, Please

Hi, ABB Bangalore. Monday November Thursday November, I\u2019m Airport Limousine Bengaluru Airport. I\u2019m
Reservation Center, Good Day, Dubai November \u2013 December Adults)
Hi Jan, Jan, Could
Hi 

In [205]:
# withoutSingularNouns = vectorizer.get_feature_names()
len(vectorizer.get_feature_names())

136

In [167]:
# withoutSingularNouns

In [168]:
# withNouns = vectorizer.get_feature_names()

In [169]:
# len(withoutNouns)

In [170]:
# len(withNouns)

In [171]:
# deleteNames = []
# for itm in withoutNouns:
#     if itm not in withoutSingularNouns:
#         deleteNames.append(itm)
# len(deleteNames)

In [172]:
# deleteNames

In [173]:
# withNouns

In [174]:
# docs = [["hello", "world", "hello"], ["goodbye", "cruel", "world"]]
# indptr = [0]
# indices = []
# data = []
# vocabulary = {}
# for d in docs:
#     for term in d:
#         index = vocabulary.setdefault(term, len(vocabulary))
#         indices.append(index)
#         data.append(1)
# indptr.append(len(indices))
# csr_matrix((data, indices, indptr), dtype=int).toarray()


In [175]:
from scipy.sparse import csr_matrix

In [197]:
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC

svc = SVC(kernel='sigmoid', gamma=1.0)
svc.fit(features_train, labels_train)
prediction = svc.predict(features_test)
accuracy_score(labels_test,prediction)

0.5

In [198]:
from sklearn.naive_bayes import MultinomialNB

mnb = MultinomialNB(alpha=0.2)
mnb.fit(features_train, labels_train)
prediction = mnb.predict(features_test)
accuracy_score(labels_test,prediction)

0.5

In [199]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(features_train, labels_train)
prediction = lr.predict(features_test)
accuracy_score(labels_test,prediction)

c:\users\lokesh\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\lokesh\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:459: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.3333333333333333

In [206]:
## grid serach CV for tuning hyperparameters
import sklearn.model_selection as modsel
param_grid_ = {'C': [1e-5, 1e-3, 1e-1, 1e0, 1e1, 1e2]}

tfidf_search = modsel.GridSearchCV(LogisticRegression(), cv=5, param_grid=param_grid_)
tfidf_search.fit(features_train, labels_train)

c:\users\lokesh\appdata\local\programs\python\python36\lib\site-packages\sklearn\model_selection\_split.py:626: Warning: The least populated class in y has only 4 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
c:\users\lokesh\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\lokesh\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:459: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
c:\users\lokesh\appdata\local\programs\python\python36\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from 

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'C': [1e-05, 0.001, 0.1, 1.0, 10.0, 100.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [207]:
# vectorizer.vocabulary_

pre = tfidf_search.predict(features_test)
accuracy_score(labels_test,pre)

0.5

### testing for the runtime sample example

In [186]:
sampleExample = """Hi

Please cancel my booking ."""

In [187]:
# sampleExample

In [188]:
textFeaturesTest = pd.Series([sampleExample])
textFeaturesTest = textFeaturesTest.apply(clean)
featuresTest = vectorizer.transform(textFeaturesTest)


In [189]:
mnb.predict(featuresTest)[0]

'Cancellation'

### writting data into processed csv files

In [190]:
cols = vectorizer.get_feature_names()
dataCsv = features.toarray()
df_to_csv = pd.DataFrame(dataCsv, columns=cols)
df_to_csv = df_to_csv.join(data["class_to_exec"], how="outer")

df_to_csv.to_csv('..\datawe\processed\data.csv')

In [191]:
vectorizer.vocabulary_

{'dear': 84,
 'rohit': 223,
 'per': 193,
 'conversation': 73,
 'ago': 10,
 'please': 198,
 'book': 46,
 'confirm': 68,
 'executive': 101,
 'room': 224,
 'taj': 256,
 'gateway': 114,
 'ganges': 113,
 'december': 86,
 'two': 269,
 'night': 179,
 'hi': 127,
 'agnes': 9,
 'received': 210,
 'response': 221,
 'email': 99,
 'yet': 282,
 'could': 78,
 'let': 151,
 'know': 147,
 'confirmed': 70,
 'corporate': 76,
 'holiday': 128,
 'plan': 197,
 'kindly': 145,
 'arrange': 24,
 'following': 108,
 'accommodation': 1,
 'hotelresort': 132,
 'detail': 92,
 'given': 117,
 'below': 42,
 'serial': 232,
 'number': 183,
 'name': 177,
 'arun': 25,
 'gadamshetty': 111,
 'no': 180,
 'department': 90,
 'spare': 244,
 'manufacturing': 164,
 'contact': 71,
 'place': 196,
 'vivanta': 275,
 'madikeri': 159,
 'coorg': 74,
 'type': 270,
 'superior': 252,
 'charm': 61,
 'check': 63,
 'datetime': 82,
 'dec': 85,
 'pm': 199,
 'comment': 66,
 'requester': 218,
 'honeymoon': 129,
 'trip': 268,
 'traveller': 266,
 'relat

In [192]:
help(TfidfVectorizer)

Help on class TfidfVectorizer in module sklearn.feature_extraction.text:

class TfidfVectorizer(CountVectorizer)
 |  Convert a collection of raw documents to a matrix of TF-IDF features.
 |  
 |  Equivalent to CountVectorizer followed by TfidfTransformer.
 |  
 |  Read more in the :ref:`User Guide <text_feature_extraction>`.
 |  
 |  Parameters
 |  ----------
 |  input : string {'filename', 'file', 'content'}
 |      If 'filename', the sequence passed as an argument to fit is
 |      expected to be a list of filenames that need reading to fetch
 |      the raw content to analyze.
 |  
 |      If 'file', the sequence items must have a 'read' method (file-like
 |      object) that is called to fetch the bytes in memory.
 |  
 |      Otherwise the input is expected to be the sequence strings or
 |      bytes items are expected to be analyzed directly.
 |  
 |  encoding : string, 'utf-8' by default.
 |      If bytes or files are given to analyze, this encoding is used to
 |      decode.
 |

In [193]:
testTfIdf = TfidfVectorizer(vocabulary=vectorizer.vocabulary_)

In [194]:
testTfIdf.fit(textFeaturesTest)
da = testTfIdf.transform(textFeaturesTest)

In [195]:
da.data

array([0.5, 0.5, 0.5, 0.5])

In [196]:
textFeaturesTest

0    hi please cancel booking
dtype: object

In [224]:
textFeatures = data['content'].copy()
textFeatures = textFeatures.apply(clean)


Please Check December January
Check December January

Rohit, Taj Gateway Ganges December
Dear Sir/Madam Re: Taj Taj Palace Hotel Taj Swarna Amritsar Taj Fort Aguada Goa. My Husband (Amrit Singh) Taj General Please
Hotel/Resort Arun Gadamshetty P Department Manufacturing Department Contact No Taj Madikeri Coorg, Coorg Room Type Charm Check Date-Time Dec PM Check Date-Time Dec PM No Honeymoon Traveller Details: Arun Gadamshetty Relation Age SHWETA GADAMSHETTY Relation Spouse Age No Please
Dear Gampa Greetings Lerner Group Please CPAI Taj Mahal, New Delhi Dec Kindly
Reservation Dept, Greetings Kuala Lumpur! DELUXE DELIGHT ROOM QUEEN BED WITH BREAKFAST Check – DECEMBER Check DECEMBER Hope


Hi Please
Dear Kindly Cancel
Dear Team, Please
Lake Palace Check December January
Dear Team Inner Circle, Taj Mahal Mumbai November Mumbai December Room Stay Inner Circle
Hi, ABB Bangalore. Monday November Thursday November, I’m Airport Limousine Bengaluru Airport. I’m
Reservation Center, Good Day, Dub

In [225]:
textFeatures

0                                 kindly book my ticket
1     please book room at lake palace check in date ...
2     kindly book room at lake palace check in date ...
3                                can you book my ticket
4     dear rohit a per our conversation while ago pl...
5     dear sirmadam re change booking under taj inne...
6     per corporate holiday plan kindly arrange to b...
7     dear gampa ji greeting of the day lerner group...
8     dear reservation dept greeting from kuala lump...
9                           i want to cancel my booking
10                              kindly cancel my ticket
11                          hi please cancel my booking
12    dear sir since there ha been change of our tra...
13    dear team please cancel the attached booking w...
14    i want to book room at lake palace check in da...
15    dear team inner circle plan to stay at the taj...
16    hi your hotel wa recommended to me by the loca...
17    dear reservation center good day would lik